In [ ]:
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py

from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy
import dipy.reconst.dti as dti
from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper
import src.tracking as tracking

from dipy.tracking.streamline import values_from_volume
import dipy.align.vector_fields as vfu
from dipy.core.sphere import Sphere
from dipy.core import subdivide_octahedron 
from scipy.spatial import KDTree
from dipy.core import subdivide_octahedron 

from dipy.tracking.local import LocalTracking
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors
from dipy.tracking.streamline import Streamlines, transform_streamlines
from dipy.tracking import metrics
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask

import numpy as np
import warnings

import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from keras.models import load_model
from keras.layers import Activation

import importlib
importlib.reload(tracking)
importlib.reload(nn_helper)
import src.tracking as tracking
import tensorflow as tf
from src.nn_helper import swish, squared_cosine_proximity_2
from src.SelectiveDropout import SelectiveDropout
import os 
import warnings

In [ ]:
streamlines = dwi_tools.loadVTKstreamlines('results_tracking/ISMRM_2015_Tracto_challenge_data//train_res100_30k_noB0InSH_step0.6_wholeBrain_b1000_csd_ismrm_gt_aggrPast_1x1x1_noUnitTension/mlp_single_sqCos2WEP_dx_1_dy_1_dz_1_dd_100_ReLU_feat_1024_depth_3_output_3_lr_0.0001_dropout_1_bn_0_unitTangent_0--0.708287reslice-False-denoising-False-200st-20mm-fa-0.vtk')

In [ ]:
importlib.reload(dwi_tools)
sl2 = dwi_tools.filterStreamlinesByCurvature(streamlines, 20)

In [ ]:
len(sl2)

In [ ]:
dwi_tools.saveVTKstreamlines(streamlines=sl2, pStreamlines='results_tracking/ISMRM_2015_Tracto_challenge_data//train_res100_30k_noB0InSH_step0.6_wholeBrain_b1000_csd_ismrm_gt_aggrPast_1x1x1_noUnitTension/mlp_single_sqCos2WEP_dx_1_dy_1_dz_1_dd_100_ReLU_feat_1024_depth_3_output_3_lr_0.0001_dropout_1_bn_0_unitTangent_0--0.708287reslice-False-denoising-False-200st-20mm-fa-0-curv20.vtk')

In [ ]:
len(sl2)

In [ ]:
importlib.reload(dwi_tools)
sl2 = dwi_tools.filterStreamlinesByCurvature(streamlines, 20)

In [ ]:
from dipy.io.streamline import load_trk
fname = '/tmp/ISMRM_2015_Tracto_challenge_data/trackts.tck'
streams, hdr = load_trk(fname)
streamlines = Streamlines(streams)

In [ ]:
streamlines = dwi_tools.loadVTKstreamlines('mlp-077534.vtk')

In [ ]:
len(streamlines_filt)

In [ ]:
streamlines_filt = dwi_tools.filterStreamlinesByLength(streamlines, 40)

In [ ]:
streamlines_filt_2 = dwi_tools.filterStreamlinesByCurvature(streamlines, 20)

In [ ]:
len(streamlines_filt_2)

In [ ]:
dwi_tools.saveVTKstreamlines(pStreamlines='mlp-077534-c20.vtk', streamlines=streamlines_filt_2)

In [ ]:
dwi_tools.saveVTKstreamlines(pStreamlines='mlp-077534-l40.vtk', streamlines=streamlines_filt)

In [ ]:
dwi_tools.visStreamlines(streamlines_filt)

In [ ]:
# Read Streamlines
from dipy.io.streamline import load_trk
fname = '/home/nico/testDouble_IC.tck'
fname2 = fname.replace('.tck','.vtk')
streams, hdr = load_trk(fname)
streamlines = Streamlines(streams)
dwi_tools.saveVTKstreamlines(streamlines=streamlines, pStreamlines=fname2)

In [ ]:
sl = np.array(streamlines[10])

In [ ]:
sl

In [ ]:
np.sum( (sl[0:-1] - sl[1:]) ** 2, axis = 1 )

Load seed mask and prepare our data

In [ ]:
fa_threshold = 0.15
#    fa_threshold = 0.2
sh_order = 8
#    b_value = 3000
stepWidth = 0.6
minimumStreamlineLength = 50 # mm

coordinateScaling = 1
useDTIPeakDirection = True
useSphericalHarmonics = True

pCaseID = 'HCP/100408'
pCaseID = 'HCP/100307'
pCaseID = 'ISMRM_2015_Tracto_challenge_data'

pModel = 'results/train_sh8_noB0InSH_step0.6_wholeBrain_b1000_2_csd_ismrm_1x1x1_noUnitTension/models/NORM_V3_mlp_single_sqCos2_dx_1_dy_1_dz_1_dd_45_ReLU_feat_512_depth_3_output_3_lr_0.0050_dropout_1_bn_0_pt_0_unitTension_0-zvFix2_54--0.923467.h5'

tracker = load_model(pModel , custom_objects={'tf':tf, 'swish':Activation(swish), 'SelectiveDropout': SelectiveDropout, 'squared_cosine_proximity_2': squared_cosine_proximity_2})

tracker.summary()

In [ ]:
useDenoising = False
b_value = 1000
# load DWI data
print('Loading dataset %s at b=%d' % (pCaseID, b_value))
bvals,bvecs,gtab,dwi,aff,t1 = dwi_tools.loadISMRMData('data/%s' % (pCaseID), denoiseData = useDenoising)
b0_mask, binarymask = median_otsu(dwi[:,:,:,0], 2, 1)

# crop DWI data
dwi_subset, gtab_subset, bvals_subset, bvecs_subset = dwi_tools.cropDatsetToBValue(b_value, bvals, bvecs, dwi)
b0_idx = bvals < 10
b0 = dwi[..., b0_idx].mean(axis=3)
dwi_singleShell = np.concatenate((dwi_subset, dwi[..., b0_idx]), axis=3)
#    dwi_singleShell_norm = dwi_tools.normalize_dwi(dwi_singleShell, b0)
bvals_singleShell = np.concatenate((bvals_subset, bvals[..., b0_idx]), axis=0)
bvecs_singleShell = np.concatenate((bvecs_subset, bvecs[b0_idx,]), axis=0)
gtab_singleShell = gradient_table(bvals=bvals_singleShell, bvecs=bvecs_singleShell, b0_threshold = 10)

In [ ]:
use2DProjection = False
useSphericalHarmonics = True

In [ ]:
if(useSphericalHarmonics):
    print('Spherical Harmonics (ours)')
    start_time = time.time()
    tracking_data = dwi_tools.get_spherical_harmonics_coefficients(bvals=bvals_subset,bvecs=bvecs_subset,sh_order=sh_order, dwi=dwi_subset, b0 = b0)
    runtime = time.time() - start_time
    print('Runtime ' + str(runtime) + 's')

if(use2DProjection):
    print('2D projection')
    start_time = time.time()
    tracking_data, resamplingSphere = dwi_tools.resample_dwi_forunet(dwi_subset, b0, bvals_subset, bvecs_subset, sh_order=8, smooth=0, mean_centering=False)
    runtime = time.time() - start_time
    print('Runtime ' + str(runtime) + 's')


print('DTI Peak Direction/odf estimation')
start_time = time.time()
dti_model = dti.TensorModel(gtab_singleShell, fit_method='LS')
dti_fit = dti_model.fit(dwi_singleShell)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')


# whole brain seeds
wholebrainseeds = seeds_from_mask(binarymask, affine=aff)
rndseeds = random_seeds_from_mask(binarymask, seeds_count=300, seed_count_per_voxel=False, affine=aff)

In [ ]:
rndseeds = random_seeds_from_mask(binarymask, seeds_count=3000, seed_count_per_voxel=False, affine=aff)

# Start Tracking 

In [ ]:
seedsToUse = rndseeds
noX = 1
noY = 1
noZ = 1
noC = 45
noTrackingSteps = 5

In [ ]:
usePreviousDirection = False

In [ ]:
importlib.reload(tracking)
warnings.filterwarnings("ignore")
start_time = time.time()
streamlines, vNorms, sl_forward, sl_backward, slp = tracking.startWithStopping__(usePreviousDirection=usePreviousDirection, printProgress = True, fa_threshold = 0, mask=binarymask,fa=dti_fit.fa, inverseDirection=False, seeds=seedsToUse, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = stepWidth, coordinateScaling = coordinateScaling, noIterations = noTrackingSteps)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

In [ ]:
np.sqrt(np.sum((streamlines[2][0:-1] - streamlines[2][1:])**2, axis=1))

In [ ]:
streamlines[2]

In [ ]:
seedsToUse[2]

In [ ]:
sl_forward[2]

In [ ]:
sl_backward[2]

In [ ]:
streamlines_mlp_simple_sc,vNorms = tracking.startWithStopping(printProgress = True, bayesianModel = False, fa_threshold = fa_threshold, mask=binarymask,fa=dti_fit.fa, bitracker=True, inverseDirection=False, seeds=seedsToUse, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = stepWidth, coordinateScaling = coordinateScaling, noIterations = noTrackingSteps)
streamlines_mlp_simple_sc_2,vNorms_2 = tracking.startWithStopping(printProgress = True, bayesianModel = False, fa_threshold = fa_threshold, mask=binarymask,fa=dti_fit.fa, bitracker=True, inverseDirection=True, seeds=seedsToUse, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = stepWidth, coordinateScaling = coordinateScaling, noIterations = noTrackingSteps)
streamlines_joined_sc = tracking.joinTwoAlignedStreamlineLists(streamlines_mlp_simple_sc,streamlines_mlp_simple_sc_2)

In [ ]:
dwi_tools.visStreamlines(streamlines_mlp_simple,dwi_singleShell[:,:,:,0], vol_slice_idx = 30)

In [ ]:
dwi_tools.visStreamlines(Streamlines(streamlines_joined_sc),dwi_singleShell[:,:,:,0], vol_slice_idx = 30)

In [ ]:
tracker.layers[6].setDropoutEnabled(1)

In [ ]:
tracker.layers[8].setDropoutEnabled(1)

In [ ]:
tracker.layers[10].setDropoutEnabled(1)

In [ ]:
def setAllDropoutLayers(m, value):
    ll = [item for item in m.layers if type(item) is SelectiveDropout]
    for ditLayer in ll:
        ditLayer.setDropoutEnabled(value)
        
def printDropoutLayersState(m):
    ll = [item for item in m.layers if type(item) is SelectiveDropout]
    for ditLayer in ll:
        print(ditLayer._getDropoutEnabled())

In [ ]:
setAllDropoutLayers(tracker, 0)
printDropoutLayersState(tracker)

In [ ]:
tracker.summary()

In [ ]:
print("Tractography")
tracking_data = dwi_singleShell_norm
fa_threshold = 0.15
showDebugOutput = True
#useBayesianTracker = True
import importlib
importlib.reload(tracking) 
import src.tracking as tracking
seedsToUse = wholebrainseeds 
#seedsToUse = rndseeds 
#seedsToUse = ccseeds 

start_time = time.time()
streamlines_mlp_simple_sc,vNorms = tracking.startWithStopping(printProgress = True, bayesianModel = useBayesianTracker, fa_threshold = fa_threshold, mask=binarymask,fa=dti_fit.fa, bitracker=useBitracker, inverseDirection=False, seeds=seedsToUse, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

start_time = time.time()
streamlines_mlp_simple_sc_2,vNorms2 = tracking.startWithStopping(printProgress = True, bayesianModel = useBayesianTracker, fa_threshold = fa_threshold, mask=binarymask,fa=dti_fit.fa, bitracker=useBitracker, inverseDirection=True, seeds=seedsToUse, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

# join left and right streamlines
print("Postprocessing streamlines and removing streamlines shorter than " + str(minimumStreamlineLength) + " mm")
streamlines_joined_sc = tracking.joinTwoAlignedStreamlineLists(streamlines_mlp_simple_sc,streamlines_mlp_simple_sc_2)

streamlines_joined_sc = dwi_tools.filterStreamlinesByLength(streamlines_joined_sc, minimumStreamlineLength)

[metrics.length(streamlines_mlp_simple_sc[0]), metrics.length(streamlines_mlp_simple_sc_2[0])]
streamlines_joined_sc = dwi_tools.filterStreamlinesByLength(streamlines_joined_sc, minimumStreamlineLength)
streamlines_joined_sc_imageCS = transform_streamlines(streamlines_joined_sc, np.linalg.inv(aff))
vNorms_joined_sc = tracking.joinTwoAlignedStreamlineLists(vNorms,vNorms2)
dwi_tools.saveVTKstreamlines(streamlines_joined_sc,pResult + '.vtk')

In [ ]:
dwi_tools.saveVTKstreamlines(streamlines_joined_sc,pResult + '.vtk')

In [ ]:
import importlib
importlib.reload(tracking) 
import src.tracking as tracking
streamlines_joined_sc = tracking.joinTwoAlignedStreamlineLists(streamlines_mlp_simple_sc,streamlines_mlp_simple_sc_2)

In [ ]:
streamlines_joined_sc[111111]

In [ ]:
streamlines_mlp_simple_sc[111111]

In [ ]:
streamlines_mlp_simple_sc_2[111111]

In [ ]:
streamlines_mlp_simple_sc[0]

# Visualize streamlines

In [ ]:
import importlib
importlib.reload(dwi_tools) 
import src.dwi_tools as dwi_tools
streamlines = dwi_tools.loadVTKstreamlines('cluster_00001.vtp') # RAS!

In [ ]:
def isFrenetSerretOk(streamline,threshold):
    T,N,B,k,t = metrics.frenet_serret(streamline)
    return np.max(k) < threshold

def filterStreamlinesByOrientation(streamlines, orientationThreshold = 0.3):
    #return [x for x in streamlines if np.max(metrics.mean_curvature(x)) < orientationThreshold]
    return [x for x in streamlines if isFrenetSerretOk(x,orientationThreshold)]

In [ ]:
sl2 = filterStreamlinesByOrientation(streamlines, 0.3)

In [ ]:
metrics.mean_curvature(streamlines)

In [ ]:
len(sl2)

In [ ]:
dwi_tools.saveVTKstreamlines(sl2,'results/sl2_k03.vtk');

In [ ]:
from dipy.tracking import metrics as tm

#[tm.mean_curvature(streamlines_joined_sc_imageCS[0]), tm.mean_orientation(streamlines_joined_sc_imageCS[0])]

In [ ]:
T,N,B,k,t = metrics.frenet_serret(streamlines[100])

In [ ]:
np.max(k)

In [ ]:
idx = 100
np.max(( (streamlines_joined_sc[idx][0:-1] - streamlines_joined_sc[idx][1:]) ** 2))

In [ ]:
import importlib
importlib.reload(dwi_tools) 
import src.dwi_tools as dwi_tools
dwi_tools.saveVTKstreamlinesWithPointdata(streamlines_joined_sc,pResult + '.vtk',vNorms_joined_sc,2) # hacky, better compute maximum from data

In [ ]:
dwi_tools.visStreamlines(streamlines_joined_sc_imageCS,t1, vol_slice_idx = 75)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(vNorms_joined_sc[0])

In [ ]:
plt.plot(streamlines_joined_sc[0])

In [ ]:
import importlib
importlib.reload(dwi_tools) 
import src.dwi_tools as dwi_tools
dwi_tools.saveVTKstreamlinesWithPointdata(streamlines_joined_sc,pResult + '-pDropout_0.5.vtk',vNorms_joined_sc,2) # hacky, better compute maximum from data

In [ ]:
dwi_tools.saveVTKstreamlines(streamlines_joined_sc,pResult + '-raw-noDropout.vtk') 

In [ ]:
tracker.layers[6].setDropoutEnabled(1)

In [ ]:
tracker.layers[6]._getDropoutEnabled()

In [ ]:
x = np.ones([1,1,1,1,108])
y = np.ones([1,3])
tracker.predict([x,y])

In [ ]:
tracker.predict([x,y])

In [ ]:
tracker.layers[6].rate

--
# ISMRM

In [ ]:
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py

from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy
import dipy.reconst.dti as dti
from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper
import src.tracking as tracking

from dipy.tracking.streamline import values_from_volume
import dipy.align.vector_fields as vfu
from dipy.core.sphere import Sphere
from dipy.core import subdivide_octahedron 
from scipy.spatial import KDTree
from dipy.core import subdivide_octahedron 

from dipy.tracking.local import LocalTracking
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors
from dipy.tracking.streamline import Streamlines, transform_streamlines
from dipy.tracking import metrics
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask

import numpy as np
import warnings

import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from keras.models import load_model
from keras.layers import Activation

import importlib
importlib.reload(tracking)
importlib.reload(nn_helper)
import src.tracking as tracking
import tensorflow as tf
from src.nn_helper import swish, squared_cosine_proximity_2
from src.SelectiveDropout import SelectiveDropout
import os 
import warnings

fa_threshold = 0.15
#fa_threshold = 0.2
sh_order = 8
b_value = 1000
minimumStreamlineLength = 40 # mm
### maximum lengths?
stepWidth = 1.0 # mm
noTrackingSteps = 200

coordinateScaling = 1
useDTIPeakDirection = True
useSphericalHarmonics = True
usePreviousDirection = True
useBayesianTracker = False
use2DProjection = False
useDenoising = False

pCaseID = 'ISMRM_2015_Tracto_challenge_data'
#pCaseID = 'ISMRM_2015_Tracto_challenge_ground_truth_dwi_v2'

pModel = 'results/train_sh8_noB0InSH_step0.6_wholeBrain_b1000_2_csd_ismrm_1x1x1_noUnitTension/models/V3_mlp_single_2_sqCos2_dx_1_dy_1_dz_1_dd_45_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_0_pt_0_unitTension_0-zvFix2_122--0.971016.h5'

pModel = 'results/train_sh8_noB0InSH_step0.6_wholeBrain_b1000_2_csd_ismrm_1x1x1_noUnitTension/models/V3_mlp_doubleIn_single_sqCos2_dx_1_dy_1_dz_1_dd_45_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_0_pt_0_unitTension_0-zvFix2_117--0.995002.h5'

pResult = "results_tracking/sw1_" + pCaseID + pModel.replace('.h5','').replace('/models/','/').replace('results','') + '-denoising-' + str(useDenoising) + '-' + str(noTrackingSteps) + 'st-fa-' + str(fa_threshold)

tracker = load_model(pModel , custom_objects={'tf':tf, 'swish':Activation(swish), 'SelectiveDropout': SelectiveDropout, 'squared_cosine_proximity_2': squared_cosine_proximity_2})

os.makedirs(pResult, exist_ok=True)

tracker.summary()

if(pModel.find("mlp_single")>0):
    usePreviousDirection = False

if(pModel.find("cnn_special")>0):
    use2DProjection = True
    useSphericalHarmonics = False
    noC = 1
    noSamples, noX, noY, noZ = tracker.get_input_shape_at(0)[0]
elif(usePreviousDirection): 
    noSamples, noX, noY, noZ, noC = tracker.get_input_shape_at(0)[0]
else: 
    noSamples, noX, noY, noZ, noC = tracker.get_input_shape_at(0)

print('Loaded model with  (dx %d, dy %d, dz %d) and %d channels' % (noX, noY, noZ, noC))

# load DWI data
print('Loading dataset %s at b=%d' % (pCaseID, b_value))
bvals,bvecs,gtab,dwi,aff,t1 = dwi_tools.loadISMRMData('data/%s' % (pCaseID), denoiseData = useDenoising, resliceToHCPDimensions=True)
b0_mask, binarymask = median_otsu(dwi[:,:,:,0], 2, 1)

# crop DWI data
dwi_subset, gtab_subset, bvals_subset, bvecs_subset = dwi_tools.cropDatsetToBValue(b_value, bvals, bvecs, dwi)
b0_idx = bvals < 10
b0 = dwi[..., b0_idx].mean(axis=3)

if(useSphericalHarmonics):
    print('Spherical Harmonics (ours)')
    start_time = time.time()
    tracking_data = dwi_tools.get_spherical_harmonics_coefficients(bvals=bvals_subset,bvecs=bvecs_subset,sh_order=sh_order, dwi=dwi_subset, b0 = b0)
    runtime = time.time() - start_time
    print('Runtime ' + str(runtime) + 's')

if(use2DProjection):
    print('2D projection')
    start_time = time.time()
    tracking_data, resamplingSphere = dwi_tools.resample_dwi_forunet(dwi_subset, b0, bvals_subset, bvecs_subset, sh_order=8, smooth=0, mean_centering=False)
    runtime = time.time() - start_time
    print('Runtime ' + str(runtime) + 's')


print('DTI Peak Direction/odf estimation')   
dwi_singleShell = np.concatenate((dwi_subset, dwi[..., b0_idx]), axis=3)
bvals_singleShell = np.concatenate((bvals_subset, bvals[..., b0_idx]), axis=0)
bvecs_singleShell = np.concatenate((bvecs_subset, bvecs[b0_idx,]), axis=0)
gtab_singleShell = gradient_table(bvals=bvals_singleShell, bvecs=bvecs_singleShell, b0_threshold = 10)

start_time = time.time()
dti_model = dti.TensorModel(gtab_singleShell, fit_method='LS')
dti_fit = dti_model.fit(dwi_singleShell)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')


# whole brain seeds
wholebrainseeds = seeds_from_mask(binarymask, affine=aff)

print("Tractography")
seedsToUse = wholebrainseeds 

warnings.filterwarnings("ignore")

In [ ]:
noTrackingSteps = 5

In [ ]:
importlib.reload(tracking)
warnings.filterwarnings("ignore")
start_time = time.time()
streamlines_joined_sc,vNorms,slf,slb = tracking.startWithStopping__(printfProfiling = False, printProgress = True, fa_threshold = fa_threshold, mask=binarymask,fa=dti_fit.fa, inverseDirection=False, seeds=seedsToUse, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = stepWidth, coordinateScaling = coordinateScaling, noIterations = noTrackingSteps, usePreviousDirection=usePreviousDirection)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

In [ ]:
len(slf)

In [ ]:
len(slb)

In [ ]:
slf[0]

In [ ]:
slb[0]

In [ ]:
streamlines_joined_sc[0]